In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
import grader

# SQL Miniproject


## Introduction


The city of New York does restaurant inspections and assigns a grade. Inspections data for the last 4 years are available on s3 as an SQLite database, which you can import in the next few cells. These were extracted from a set of csv files and an xls file, as described in the <b>How we loaded the data</b> section


The raw data can be found [here](https://s3.amazonaws.com/dataincubator-course/coursedata/nyc_inspection_data.zip) and can be useful to look at. The file `RI_Webextract_BigApps_Latest.xls` contains a description of each of the datafiles and what the columns mean.

In [3]:
!aws s3 sync s3://dataincubator-course/coursedata/ . --exclude '*' --include 'nyc_inspection.db'

download: s3://dataincubator-course/coursedata/nyc_inspection.db to ./nyc_inspection.db


In [4]:
#This will load the pre-existing tables
%load_ext sql
%sql sqlite:///nyc_inspection.db

/opt/conda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


u'Connected: None@nyc_inspection.db'

To see what tables are in the database:

In [5]:
%%sql
SELECT * FROM sqlite_master WHERE "type"='table';

Done.


type,name,tbl_name,rootpage,sql
table,writer,writer,2,"CREATE TABLE writer (first_name, last_name, year)"
table,webextract,webextract,3,"CREATE TABLE webextract ( ""index"" BIGINT, ""CAMIS"" BIGINT, ""DBA"" TEXT, ""BORO"" BIGINT, ""BUILDING"" TEXT, ""STREET"" TEXT, ""ZIPCODE"" FLOAT, ""PHONE"" TEXT, ""CUISINECODE"" BIGINT, ""INSPDATE"" TEXT, ""ACTION"" TEXT, ""VIOLCODE"" TEXT, ""SCORE"" FLOAT, ""CURRENTGRADE"" TEXT, ""GRADEDATE"" TEXT, ""RECORDDATE"" TEXT)"
table,violations,violations,24393,"CREATE TABLE violations ( ""index"" BIGINT, ""STARTDATE"" TEXT, ""ENDDATE"" TEXT, ""CRITICALFLAG"" TEXT, ""VIOLATIONCODE"" TEXT, ""VIOLATIONDESC"" TEXT)"
table,cuisine,cuisine,24426,"CREATE TABLE cuisine ( ""index"" BIGINT, ""CUISINECODE"" BIGINT, ""CODEDESC"" TEXT)"
table,boroughs,boroughs,24428,"CREATE TABLE boroughs ( id INT, name TEXT)"


And to look at the format of an individual table (note that you may need to change types to get the answers in the right form):

In [6]:
#The PRAGMA statement is an SQL extension specific to SQLite
%%sql
PRAGMA table_info(webextract)

Done.


cid,name,type,notnull,dflt_value,pk
0,index,BIGINT,0,None,0
1,CAMIS,BIGINT,0,None,0
2,DBA,TEXT,0,None,0
3,BORO,BIGINT,0,None,0
4,BUILDING,TEXT,0,None,0
5,STREET,TEXT,0,None,0
6,ZIPCODE,FLOAT,0,None,0
7,PHONE,TEXT,0,None,0
8,CUISINECODE,BIGINT,0,None,0
9,INSPDATE,TEXT,0,None,0


In [8]:
!sqlite3 cmd "PRAGMA table_info(webextract)"

## SQLite3


The project should be written in SQL. Between SQLite and PostgreSQL we recommend sqlite3 for this project.  You can use the sqlite command prompt by running this command in bash
```bash
sqlite3 cmd "DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"
```
Alternatively, you can run bash commands in a jupyter notebook by prepending the `!` in a code cell (notice that we conveniently get the output displayed

In [ ]:
!sqlite3 cmd """\
DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"""

Finally, we use the [ipython-sql extension](https://github.com/catherinedevlin/ipython-sql#ipython-sql) by first loaidng the sql extension and then running our code with the "magic" command in the first line
```python
%%sql sqlite://
```
Notice that the output table is formatted nicely as a nice HTML table.

This is our recommended technique.  However, the grader is expecting python objects and you may need to use list comprehensions to reformat this output

In [ ]:
%%sql 
DROP TABLE IF EXISTS writer;
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);

SELECT * FROM writer;

In [ ]:
result = _
#This captures the output of the previous cell

In [ ]:
result

## How we loaded the data


For future reference, here is how you can load data in to SQL (with examples).  If you have a csv file you created with something like

```
!printf "Name,Age\nAlice,3\nBob,10" > sample.csv.nogit
```


Then Sqlite3 has a convenient [`.import` function](https://sqlite.org/cli.html#csv_import) which can create tables from `.csv` files.

```bash
sqlite> .import sample.csv.nogit sample
sqlite> SELECT * FROM sample;
```

The files may contain malformatted text.  Unfortunately, this is all too common.  As a stop gap, remember that [`iconv`](https://linux.die.net/man/1/iconv) is a unix utility that can convert files between different text encodings.

Alternatively, you can also read csv files using pandas and convert that into SQL via some sql magic (this is what we actually did).

```
import pandas as pd
sample = pd.read_csv('sample.csv.nogit')
%sql DROP TABLE IF EXISTS sample
%sql PERSIST sample
%sql SELECT * FROM sample;
```

## Question 1: null_entries


Return the number of inspections (CAMIS, INSPDATE pairs) that do not have a score - i.e. where none of the rows with those (CAMIS, INSPDATE) values has a score. Remove the corresponding rows from the dataset for the rest of the questions in the assignment.

**Question:** How else might we have handled this?

In [ ]:
def null_entries():
    return 0

grader.score('sql__null_entries', null_entries)

## Question 2: score_by_zipcode


Return a list of tuples of the form:

    (zipcode, mean score, number of restaurants)

for each of the 87 zipcodes in the city with over 100 restaurants. Use the score from the latest inspection date for each restaurant. Sort the list in ascending order by mean score.

**Note:** There is an interesting discussion here about what the mean score *means* in this dataset. Think about what we're actually calculating - does it represent what we're trying to understand about these zipcodes?

What if we use the average of a restaurant's inspections instead of the latest?

**Checkpoints:**
- Total unique restaurants: 24,361;
- Total restaurants in valid zipcodes: 19,172


In [ ]:
def score_by_zipcode():
    return [("11201", 9.81739130434783, 345)] * 87

grader.score('sql__score_by_zipcode', score_by_zipcode)

## Question 3: score_by_map


The above are not terribly enlightening.  Use [CartoDB](http://cartodb.com/) to produce a map of average scores by zip code.  You can sign up for a free trial.

You will have to use their wizard to plot the data by [zipcode](https://carto.com/learn/guides/analysis/georeference). You will need to specify "USA" in the countryfield.  Then use the "share" button to return a link of the form [https://x.cartodb.com/](https://x.cartodb.com/).

**For fun:** How do JFK, Brighton Beach, Liberty Island (home of the Statue of Liberty), Financial District, Chinatown, and Coney Island fare?

**For more fun:** Plot restaurants as pins on the map, allowing the user to filter by "low", "middling", or "high"-scoring restaurants. You can use a CASE WHEN statement to create the different groups based on score thresholds.

In [ ]:
def score_by_map():
    # must be url of the form https://x.cartodb.com/...
    return "https://cartodb.com"

grader.score('sql__score_by_map', score_by_map)

## Question 4: score_by_borough

Return a list of tuples of the form:

    (borough, mean score, number of restaurants)

for each of the city's five boroughs. Use the latest score for each restaurant. Sort the list in ascending order by grade.

**Hint:** You will have to perform a join with the `boroughs` table. The borough names should be reported in ALL CAPS.

**Checkpoint:**
- Total restaurants in valid boroughs: 24,350

In [ ]:
def score_by_borough():
    return [("MANHATTAN", 10.7269875502402, 10201)] * 5

grader.score('sql__score_by_borough', score_by_borough)

## Question 5: violations_by_cuisine


We want to look at violations themselves now.  We'll need to think more carefully about what we're measuring, since most restaurants have many inspections with possibly multiple violations per inspection, or long stretches of inspections with no violations.

There are many ways to deal with this normalization issue, but we'll go with a fairly straightforward one: dividing the number of violations by the length of time (in years) the restaurant has been open.  As a proxy for the length, we'll look at the difference between the oldest and newest inspection date, treating anything less than 30 days as 30 days (to account for those that were only inspected once, we'll assume everything was open for at least a month).

Since there are so many restaurants, we'll group them by cuisine and do a weighted average by computing 

    (total violations for a cuisine) / (total restaurant-years for that cuisine)

Return a list of 75 tuples of the form

    (cuisine name, reports per restaurant-year)
    
for cuisines with at least 100 violations total, ordered by increasing number of reports per restaurant-year
    
**Note:** This isn't the only way to normalize things.  How would other ways affect the computation?  If you similarly wanted to compute an average score by cuisine, how might you go about doing that?
    
**Checkpoint:**
- Total entries from valid cuisines: 522,410

In [ ]:
def score_by_cuisine():
    return [("French", 20.3550686378036)] * 75

grader.score('sql__score_by_cuisine', score_by_cuisine)

## Question 6: specific_violations_by_cuisine

Which cuisines tend to have a disproportionate number of what which violations? Answering this question isn't easy becuase you have to think carefully about normalizations.

1. More popular cuisine categories will tend to have more violations just becuase they represent more restaurants.
2. Similarly, some violations are more common.  For example, knowing that "Equipment not easily movable or sealed to floor" is a common violation for Chinese restuarants is not particularly helpful when it is a common violation for all restaurants.

The right quantity is to look at is the conditional probability of a specific type of violation given a specific cuisine type and divide it by the unconditional probability of the violation for the entire population. Taking this ratio gives the right answer.  Return the 20 highest ratios of the form:

    ((cuisine, violation), ratio, count)

**Hint:**
1. You might want to check out this [Stackoverflow post](http://stackoverflow.com/questions/972877/calculate-frequency-using-sql).
2. The definition of a violation changes with time.  For example, 10A can mean two different things "Toilet facility not maintained ..." or "Vermin or other live animal present ..." when things were prior to 2003. To deal with this, you should limit your analysis to violation codes with end date after Jan 1, 2014. (This end date refers to the validity time ranges in Violation.txt).
3. The ratios don't mean much when the number of violations of a given type and for a specific category are not large (why not?).  Be sure to filter these out.  We chose 100 as our cutoff.

**Checkpoint:**
- Top 20 ratios mean: 2.360652529900757

In [ ]:
def violation_by_cuisine():
    return [(("Café/Coffee/Tea",
              "Toilet facility not maintained and provided with toilet paper; "
              "waste receptacle and self-closing door."),
             1.87684775827172, 315)] * 20

grader.score('sql__violation_by_cuisine', violation_by_cuisine)

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*